# 🧠 Robust DistilBERT Phishing Detector (70/15/15 Split)
This notebook builds a phishing email detector using DistilBERT.
- 70% Train, 15% Validation, 15% Test
- W&B logging
- Ready for advanced enhancements


In [1]:
!pip install -q transformers datasets wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12=

In [2]:
from datasets import load_dataset
ds = load_dataset('zefang-liu/phishing-email-dataset')
df = ds['train'].to_pandas()
df['Email Type'] = df['Email Type'].str.lower().str.strip()
df['label'] = df['Email Type'].map({'phishing email': 1, 'safe email': 0})
df = df.dropna(subset=['Email Text', 'label'])
df['text'] = df['Email Text'].astype(str).str.strip()
df = df[df['text'].str.len() > 10]
df = df[['text', 'label']]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/616 [00:00<?, ?B/s]

Phishing_Email.csv:   0%|          | 0.00/52.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18650 [00:00<?, ? examples/s]

## 🔀 70-15-15 Split: Train / Validation / Test


In [3]:
from sklearn.model_selection import train_test_split
# First split: train vs temp (val + test)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.3, random_state=42)

# Second split: validation vs test (15/15)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42)


In [4]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
import torch
class PhishingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = PhishingDataset(train_encodings, train_labels)
val_dataset = PhishingDataset(val_encodings, val_labels)
test_dataset = PhishingDataset(test_encodings, test_labels)

In [6]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import wandb

# 🧹 Clean up any previous runs
wandb.finish()
wandb.login()

# ✅ Load model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# ✅ Use memory-efficient padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ✅ Optimized training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # 🔽 Lowered to reduce RAM
    per_device_eval_batch_size=16,  # 🔽 Safer eval batch size
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=200,
    load_best_model_at_end=True,
    fp16=True,  # ✅ Mixed precision = huge RAM save
    disable_tqdm=True,
    report_to="none",  # 🔕 Optional: turn off W&B to test first
)

# ✅ Safety checks
if train_dataset is None or len(train_dataset) == 0:
    raise ValueError("train_dataset is empty or not loaded properly.")
if val_dataset is None or len(val_dataset) == 0:
    raise ValueError("val_dataset is empty or not loaded properly.")

# ✅ Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# ✅ Train
trainer.train()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: akataship (akataship-models) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.2341, 'grad_norm': 50.4580192565918, 'learning_rate': 4.793640766477153e-05, 'epoch': 0.12634238787113075}
{'loss': 0.1326, 'grad_norm': 0.026492709293961525, 'learning_rate': 4.583070120025269e-05, 'epoch': 0.2526847757422615}
{'loss': 0.1135, 'grad_norm': 0.03200073912739754, 'learning_rate': 4.3724994735733844e-05, 'epoch': 0.3790271636133923}
{'loss': 0.1032, 'grad_norm': 3.3850014209747314, 'learning_rate': 4.161928827121499e-05, 'epoch': 0.505369551484523}
{'loss': 0.0923, 'grad_norm': 0.05725761502981186, 'learning_rate': 3.951358180669615e-05, 'epoch': 0.6317119393556538}
{'loss': 0.0959, 'grad_norm': 0.03591373190283775, 'learning_rate': 3.74078753421773e-05, 'epoch': 0.7580543272267846}
{'loss': 0.0535, 'grad_norm': 0.8122077584266663, 'learning_rate': 3.5302168877658455e-05, 'epoch': 0.8843967150979154}
{'eval_loss': 0.054339464753866196, 'eval_runtime': 10.3626, 'eval_samples_per_second': 261.904, 'eval_steps_per_second': 16.405, 'epoch': 1.0}
{'loss': 0.0776, 'g

TrainOutput(global_step=4749, training_loss=0.050071403914487944, metrics={'train_runtime': 626.8911, 'train_samples_per_second': 60.604, 'train_steps_per_second': 7.575, 'train_loss': 0.050071403914487944, 'epoch': 3.0})

In [7]:
from sklearn.metrics import classification_report
preds = trainer.predict(test_dataset)
y_pred = preds.predictions.argmax(-1)
print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1677
           1       0.99      0.99      0.99      1037

    accuracy                           0.99      2714
   macro avg       0.99      0.99      0.99      2714
weighted avg       0.99      0.99      0.99      2714



## ✅ Final Optimized Training Block with W&B and Crash-Safe Settings

In [ ]:

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import wandb

# 🧹 Clean up any previous runs
wandb.finish()
wandb.login()

# ✅ Load model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# ✅ Use memory-efficient padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ✅ Optimized training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=200,
    load_best_model_at_end=True,
    fp16=True,
    disable_tqdm=True,
    report_to="wandb"
)

# ✅ Safety checks
if train_dataset is None or len(train_dataset) == 0:
    raise ValueError("train_dataset is empty or not loaded properly.")
if val_dataset is None or len(val_dataset) == 0:
    raise ValueError("val_dataset is empty or not loaded properly.")

# ✅ Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# ✅ Train
trainer.train()
